In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection  import train_test_split
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
from sklearn import metrics

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



/kaggle/input/water-potability/water_potability.csv
/kaggle/input/breast-cancer-coimbra-data-set/dataR2.csv


# Veri Ön İşlemesi

In [2]:
df = pd.read_csv('../input/water-potability/water_potability.csv')
print(df)

            ph    Hardness        Solids  Chloramines     Sulfate  \
0          NaN  204.890455  20791.318981     7.300212  368.516441   
1     3.716080  129.422921  18630.057858     6.635246         NaN   
2     8.099124  224.236259  19909.541732     9.275884         NaN   
3     8.316766  214.373394  22018.417441     8.059332  356.886136   
4     9.092223  181.101509  17978.986339     6.546600  310.135738   
...        ...         ...           ...          ...         ...   
3271  4.668102  193.681735  47580.991603     7.166639  359.948574   
3272  7.808856  193.553212  17329.802160     8.061362         NaN   
3273  9.419510  175.762646  33155.578218     7.350233         NaN   
3274  5.126763  230.603758  11983.869376     6.303357         NaN   
3275  7.874671  195.102299  17404.177061     7.509306         NaN   

      Conductivity  Organic_carbon  Trihalomethanes  Turbidity  Potability  
0       564.308654       10.379783        86.990970   2.963135           0  
1       592.88535

## b)Veri setinde tanımlı olmayan değerler bulmaya çalışıyoruz

In [3]:
df.isnull().sum()

ph                 491
Hardness             0
Solids               0
Chloramines          0
Sulfate            781
Conductivity         0
Organic_carbon       0
Trihalomethanes    162
Turbidity            0
Potability           0
dtype: int64

## eksik verileri ortalama değere göre dolduruyoruz.

In [4]:
print(df.mean())
df_mean = df.fillna(df.mean())
print(df)
print(df_mean)


ph                     7.080795
Hardness             196.369496
Solids             22014.092526
Chloramines            7.122277
Sulfate              333.775777
Conductivity         426.205111
Organic_carbon        14.284970
Trihalomethanes       66.396293
Turbidity              3.966786
Potability             0.390110
dtype: float64
            ph    Hardness        Solids  Chloramines     Sulfate  \
0          NaN  204.890455  20791.318981     7.300212  368.516441   
1     3.716080  129.422921  18630.057858     6.635246         NaN   
2     8.099124  224.236259  19909.541732     9.275884         NaN   
3     8.316766  214.373394  22018.417441     8.059332  356.886136   
4     9.092223  181.101509  17978.986339     6.546600  310.135738   
...        ...         ...           ...          ...         ...   
3271  4.668102  193.681735  47580.991603     7.166639  359.948574   
3272  7.808856  193.553212  17329.802160     8.061362         NaN   
3273  9.419510  175.762646  33155.578218    

## Veri normalize ediliyor

In [5]:
scaler = MinMaxScaler()
data = scaler.fit_transform(df_mean)
data = pd.DataFrame(data = data,columns = df_mean.columns)
data.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,0.505771,0.571139,0.336096,0.543891,0.680385,0.669439,0.313402,0.699753,0.286091,0.0
1,0.265434,0.297400,0.300611,0.491839,0.581699,0.719411,0.497319,0.450999,0.576793,0.0
2,0.578509,0.641311,0.321619,0.698543,0.581699,0.414652,0.562017,0.532866,0.303637,0.0
3,0.594055,0.605536,0.356244,0.603314,0.647347,0.317880,0.622089,0.808065,0.601015,0.0
4,0.649445,0.484851,0.289922,0.484900,0.514545,0.379337,0.358555,0.253606,0.496327,0.0


## Test ve train datası için kolonları ayrılır.

In [6]:
x = data.iloc[:,0:9]
y = data.iloc[:,-1]


## Temel bileşen analizine normalize edilmiş veri uygulanır.

In [7]:
pca = PCA(0.90) 
pca.fit(x)
x_pca = pca.transform(x)

## Temel bileşen analizi sonrasında kolon sayımız

In [8]:
x_pca.shape

(3276, 8)

## Test ve train datası ayrılıyor

In [9]:
x_train,x_test,y_train,y_test = train_test_split(x_pca,y,test_size=0.2,random_state=400)
print(y_train)
print(y_test)

2570    0.0
260     1.0
608     0.0
2852    1.0
2075    0.0
       ... 
1011    0.0
140     0.0
953     0.0
2366    1.0
348     1.0
Name: Potability, Length: 2620, dtype: float64
1060    0.0
623     0.0
28      0.0
1867    0.0
2339    1.0
       ... 
511     0.0
1757    0.0
1479    0.0
1985    1.0
2798    1.0
Name: Potability, Length: 656, dtype: float64


## KNN uygulanıyor.

In [10]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train,y_train.ravel())

KNeighborsClassifier()

# Sonuç


In [11]:
result = knn.predict(x_test)
print(metrics.classification_report(y_test,result))

              precision    recall  f1-score   support

         0.0       0.65      0.72      0.68       414
         1.0       0.41      0.33      0.37       242

    accuracy                           0.58       656
   macro avg       0.53      0.53      0.52       656
weighted avg       0.56      0.58      0.57       656

